In [1]:
import pandas as pd

In [2]:
def fix_smiles(smi_in):
    return smi_in.strip('[]').strip("'")

In [3]:
df_list = []
for split in ["random","scaffold"]:
    for task in ["st","mt"]:
        for i in range(0,25):
            dirname = f"data/Sol_repeated{i:03d}"
            if task == "st":
                df_test = pd.read_csv(f"{dirname}/{split}_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_ST/test_preds.csv")
            else:
                df_test = pd.read_csv(f"{dirname}/{split}_mt_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_MT/test_preds.csv")
            df_pred['SMILES'] = df_pred.smiles.apply(fix_smiles)
            df_pred.drop('smiles',axis=1,inplace=True)
            df_pred.rename(columns={"Sol": "Sol_pred"},inplace=True)
            df_test.Sol = df_test.Sol.astype(float)
            df_combo = pd.concat([df_test[["SMILES","Name","Sol"]],df_pred[["Sol_pred"]]],axis=1)
            df_combo['cv_cycle'] = i
            df_combo['split'] = split
            df_combo['method'] = f"chemprop_{task}"
            df_combo = df_combo[["cv_cycle","split","method","SMILES","Name","Sol","Sol_pred"]]
            df_list.append(df_combo)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Sol_repeated001/random_test_001.csv'

In [31]:
df_out = pd.concat(df_list)
df_out = df_out.dropna()
df_out.to_csv("Sol_repeated_regression_preds.csv",index=False)

In [32]:
df_out

,cv_cycle,split,method,SMILES,Name,Sol,Sol_pred
0,0,random,chemprop_st,CCOc1cc2nn(CCC(C)(C)O)cc2cc1NC(=O)c1cccc(C(F)F)n1,Mol2,0.928591,1.238917
1,0,random,chemprop_st,C=CC(=O)N1CCC(CNc2ncnc(N)c2-c2ccc(Oc3ccccc3)cc...,Mol24,1.371334,1.766110
2,0,random,chemprop_st,C=CC(=O)N1CCC[C@H](n2c(=O)n(-c3ccc(Oc4ccccc4)c...,Mol35,2.091447,1.427550
3,0,random,chemprop_st,CCc1nc(C)cn2nc(-c3cc(=O)n4cc(C5CCN(C)CC5)cc(C)...,Mol45,1.627714,1.673194
4,0,random,chemprop_st,Cc1ncc(CN2CC[C@@H](Nc3cncc(Cl)n3)[C@@H](C)C2)s1,Mol51,2.193871,2.186510
...,...,...,...,...,...,...,...
432,24,scaffold,chemprop_mt,CCN1C(=O)C(=O)N(Cc2sc3ccccc3c2Cl)C1=O,Mol3481,-0.031776,0.988930
433,24,scaffold,chemprop_mt,O=C(Nc1nc2ccccc2[nH]1)c1cc(Cl)ccc1Cl,Mol3483,0.254426,0.498820
434,24,scaffold,chemprop_mt,Cc1cnc(NC(=O)Cc2coc3cc(C)c(C)cc23)s1,Mol3487,0.027475,1.286758
435,24,scaffold,chemprop_mt,CNC(=O)c1ccc(-c2nnc(-c3cccc(C)c3)o2)cc1,Mol3491,-0.145131,1.389679


In [2]:
df_chemprop = pd.read_csv("Sol_repeated_regression_preds.csv")
df_lightgbm = pd.read_csv("lightgbm_repeated_regression_results.csv")
pd.concat([df_chemprop, df_lightgbm]).to_csv("Sol_regression_ChemProp_LightGBM_v2.csv",index=False)